## Currency Conversion 
    - using API

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
# from langchain_core.tools import tool, InjectedToolArg
# from typing import Annotated
from langchain_core.messages import HumanMessage
import requests

from dotenv import load_dotenv
load_dotenv()

True

### Tool Creation

In [3]:
# Tool Create

from langchain_core.tools import InjectedToolArg, tool
from typing import Annotated

@tool #tool1
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool  # tool 2
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [4]:
# Checking arguments of the Secong tool
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [5]:
# Testing First tool
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756252801,
 'time_last_update_utc': 'Wed, 27 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1756339201,
 'time_next_update_utc': 'Thu, 28 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.6692}

In [6]:
# Testing the second tool (basically it's a function, so just calling a function using .invoke since it's a llm tool now!)
convert.invoke({'base_currency_value':10, 'conversion_rate':87.6692})

876.692

### Tool Binding

In [7]:
# tool binding

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [8]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert]) # binding all the tools, it's a list we can add remove and define which to use and which to remove!

In [9]:
messages = [HumanMessage('What is the conversion rate between INR and USD, and then convert 10 USD to INR')] # creating a list to store conversion

In [10]:
messages

[HumanMessage(content='What is the conversion rate between INR and USD, and then convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [11]:
ai_message = llm_with_tools.invoke(messages)
messages.append(ai_message)
print(ai_message.tool_calls) # making AI read and respond to Human message

[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': '91495ec6-f2d0-489c-8286-60fff4413333', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10.0}, 'id': '127150d9-84c5-45c2-828f-44642f812e19', 'type': 'tool_call'}]


In [12]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'INR'},
  'id': '91495ec6-f2d0-489c-8286-60fff4413333',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10.0},
  'id': '127150d9-84c5-45c2-828f-44642f812e19',
  'type': 'tool_call'}]

[{'name': 'convert', 'args': {'base_currency_value': 10.0}, 'id': '94b25e67-bb1d-40c9-9d91-9eedd6986874', 'type': 'tool_call'}]
